In [1]:
%reset -f

In [2]:
import pandas as pd
from sssom.parsers import parse_sssom_table
from sssom import compare_dataframes
from sssom.parsers import split_dataframe
from sssom.util import MappingSetDataFrame
from os.path import join
from oaklib import OntologyResource
from oaklib.implementations import SqlImplementation
import textdistance

In [3]:
lexmatch_file = "../mappings/mondo-sources-all-lexical.sssom.tsv"
mondo_sssom = "../ontology/tmp/mondo.sssom.tsv"
dir_name = "dataframes"
exclusion_files = [
    "../ontology/reports/doid_term_exclusions.txt",
    "../ontology/reports/omim_term_exclusions.txt",
    "../ontology/reports/ordo_term_exclusions.txt",   
    "../ontology/reports/ncit_term_exclusions.txt",
    "../ontology/reports/icd10cm_term_exclusions.txt"
]


In [4]:
# Functions

def add_distance(df, col_name, txt_dist_pkg):
    df\
    .insert(\
            len(df.columns),\
            col_name,\
            df.apply\
            (\
                lambda x: txt_dist_pkg\
                 (\
                    x.subject_label.lower(), x.object_label.lower()\
                    if pd.notnull(x.object_label) else "99"\
                 ),\
                axis=1,\
            )\
           )

def print_prefixes(df):
    object_prefixes = df['object_id'].str.split(':').apply(lambda x: x[0] ).drop_duplicates()
    subject_prefixes = df['subject_id'].str.split(':').apply(lambda x: x[0] ).drop_duplicates()
    predicate_ids = df['predicate_id'].drop_duplicates()
    

    print(f"subject_prefixes:\n {subject_prefixes} \n \
          object_prefixes:\n {object_prefixes} \n \
          predicate_ids:\n {predicate_ids} ")

    
def flip_predicate(predicate_id):
    flip_dict = {
        "skos:closeMatch": "skos:closeMatch",
        "skos:relatedMatch": "skos:relatedMatch",
        "skos:narrowMatch" : "skos:broadMatch",
        "skos:broadMatch" : "skos:narrowMatch",
        "skos:exactMatch" : "skos:exactMatch"
    }
    
    return flip_dict[predicate_id]

def compare_and_comment_df(mondo_df, lex_df):
    df = compare_dataframes(mondo_df, lex_df).combined_dataframe
    df['comment'] = df['comment'].str.replace('UNIQUE_1', "MONDO_MAPPINGS")
    df['comment'] = df['comment'].str.replace('UNIQUE_2', "LEXMATCH")
    return df

def get_unmapped_df(comparison_df):
#     mappings = ["LEXMATCH", "MONDO_MAPPINGS"]
#     unmapped_df = comparison_df[
#         (comparison_df['comment'].str.contains("|".join(mappings)))
#     ]
    unmapped_lex_df = comparison_df[
        comparison_df['object_id'].str.contains("|".join(in_lex_but_not_mondo_list)) &
        comparison_df['comment'].str.contains("LEXMATCH")
    ]
    
    unmapped_mondo_df = comparison_df[
        comparison_df['object_id'].str.contains("|".join(in_mondo_but_not_lex_list)) &
        comparison_df['comment'].str.contains("MONDO_MAPPINGS")
    ]
    
    new_df = pd.concat([unmapped_lex_df, unmapped_mondo_df], axis=0)
    return new_df

def export_unmatched_exact(unmapped_df, match_type, fn):
        
    unmapped_exact = unmapped_df[(unmapped_df['comment'] == match_type) & (unmapped_df['predicate_id'] == 'skos:exactMatch')]
    unmapped_exact = replace_by_mondo_preds(unmapped_exact)
    unmapped_exact.loc[-1] = [
                                "ID",
                                "", 
                                "A oboInOwl:hasDbXref",
                                ">A oboInOwl:source",
                                ">A sssom:object_label",
                                "",
                                "",
                                "",
                                "",
                                "",
                                "",
                                ""
                             ]
    
    unmapped_exact.index += 1
    unmapped_exact = unmapped_exact.sort_index()
    unmapped_exact.to_csv(join(dir_name, fn), sep='\t', index = False)
    return unmapped_exact.head()

def make_msdf(df, prefix_map, meta):
    combined_msdf = MappingSetDataFrame(df=df, prefix_map=prefix_map, metadata=meta)
    return combined_msdf

def mapped_curie_list(df):
    all_id_df = pd.DataFrame(pd.concat([df['subject_id'], df['object_id']]))\
                .drop_duplicates(ignore_index=True)
    all_id_df = all_id_df[~all_id_df[0].str.startswith('MONDO')]
    return all_id_df[0].to_list()

def replace_by_mondo_preds(df):
    mondo_codes = {
        "skos:exactMatch": "MONDO:equivalentTo",
        "skos:relatedMatch": "MONDO:relatedTo",
        "skos:narrowMatch": "MONDO:mondoIsBroaderThanSource",
        "skos:broadMatch": "MONDO:mondoIsNarrowerThanSource",
    }
    df['predicate_id'] = df['predicate_id'].apply(lambda x: mondo_codes[x] if x in mondo_codes else x)
    return df
    

In [5]:
%%time
msdf_lex = parse_sssom_table(lexmatch_file)
msdf_mondo = parse_sssom_table(mondo_sssom)

# Use OAK to get 'object_label'
ontology_resource = OntologyResource(slug='../ontology/tmp/merged.db', local=True)
oi = SqlImplementation(ontology_resource)

CPU times: user 3min 39s, sys: 3.32 s, total: 3min 43s
Wall time: 3min 44s


In [6]:
%%time
exclusion_list = []

for file_path in exclusion_files:
    with open(file_path) as f_input:
        exclusion_list.extend(f_input.read().split('\n'))
        
exclusion_list

CPU times: user 17.6 ms, sys: 2.78 ms, total: 20.4 ms
Wall time: 21.1 ms


['DOID:0040001',
 'DOID:0040002',
 'DOID:0040003',
 'DOID:0040004',
 'DOID:0040005',
 'DOID:0040006',
 'DOID:0040007',
 'DOID:0040008',
 'DOID:0040009',
 'DOID:0040010',
 'DOID:0040011',
 'DOID:0040012',
 'DOID:0040013',
 'DOID:0040014',
 'DOID:0040015',
 'DOID:0040016',
 'DOID:0040017',
 'DOID:0040018',
 'DOID:0040019',
 'DOID:0040020',
 'DOID:0040021',
 'DOID:0040022',
 'DOID:0040023',
 'DOID:0040024',
 'DOID:0040025',
 'DOID:0040026',
 'DOID:0040027',
 'DOID:0040028',
 'DOID:0040029',
 'DOID:0040030',
 'DOID:0040031',
 'DOID:0040032',
 'DOID:0040033',
 'DOID:0040034',
 'DOID:0040035',
 'DOID:0040036',
 'DOID:0040037',
 'DOID:0040038',
 'DOID:0040040',
 'DOID:0040041',
 'DOID:0040042',
 'DOID:0040043',
 'DOID:0040044',
 'DOID:0040045',
 'DOID:0040046',
 'DOID:0040047',
 'DOID:0040048',
 'DOID:0040049',
 'DOID:0040050',
 'DOID:0040051',
 'DOID:0040052',
 'DOID:0040053',
 'DOID:0040054',
 'DOID:0040055',
 'DOID:0040056',
 'DOID:0040057',
 'DOID:0040058',
 'DOID:0040059',
 'DOID:0040061

In [7]:
%%time
all_lex_ids = mapped_curie_list(msdf_lex.df)
all_mondo_ids = mapped_curie_list(msdf_mondo.df)

in_lex_but_not_mondo_list = [x for x in all_lex_ids if x not in all_mondo_ids and x not in exclusion_list]
in_mondo_but_not_lex_list = [x for x in all_mondo_ids if x not in all_lex_ids and x not in exclusion_list]


CPU times: user 11min 39s, sys: 1.7 s, total: 11min 40s
Wall time: 11min 42s


In [8]:
in_lex_but_not_mondo_list

['DOID:0060042',
 'DOID:0060160',
 'DOID:0060204',
 'DOID:0070020',
 'DOID:0070141',
 'DOID:0070212',
 'DOID:0070309',
 'DOID:0070311',
 'DOID:0070326',
 'DOID:0070327',
 'DOID:0070331',
 'DOID:0070333',
 'DOID:0070341',
 'DOID:0070343',
 'DOID:0070348',
 'DOID:0070349',
 'DOID:0070351',
 'DOID:0070358',
 'DOID:0070359',
 'DOID:0070360',
 'DOID:0070361',
 'DOID:0070362',
 'DOID:0080380',
 'DOID:0080390',
 'DOID:0080407',
 'DOID:0080410',
 'DOID:0080411',
 'DOID:0080503',
 'DOID:0080511',
 'DOID:0080546',
 'DOID:0080574',
 'DOID:0080594',
 'DOID:0080597',
 'DOID:0080602',
 'DOID:0080606',
 'DOID:0080607',
 'DOID:0080610',
 'DOID:0080612',
 'DOID:0080638',
 'DOID:0080641',
 'DOID:0080644',
 'DOID:0080645',
 'DOID:0080646',
 'DOID:0080647',
 'DOID:0080650',
 'DOID:0080651',
 'DOID:0080652',
 'DOID:0080661',
 'DOID:0080662',
 'DOID:0080663',
 'DOID:0080664',
 'DOID:0080665',
 'DOID:0080666',
 'DOID:0080669',
 'DOID:0080670',
 'DOID:0080671',
 'DOID:0080672',
 'DOID:0080673',
 'DOID:0080674

In [9]:
pd.DataFrame(msdf_mondo.df['object_id'].str.split(':').apply(lambda x: x[0] )).value_counts()

object_id
UMLS         16619
DOID          9812
Orphanet      9779
OMIM          9310
SCTID         9043
MESH          8100
NCIT          6905
ICD10CM       2020
OMIMPS         533
ICD10WHO        18
MedDRA          12
MEDGEN           2
dtype: int64

In [10]:
print_prefixes(msdf_mondo.df)

subject_prefixes:
 0    MONDO
Name: subject_id, dtype: object 
           object_prefixes:
 0           SCTID
1            MESH
2            DOID
3        Orphanet
4            NCIT
5            UMLS
11         OMIMPS
51        ICD10CM
172          OMIM
604        MedDRA
657      ICD10WHO
25506      MEDGEN
Name: object_id, dtype: object 
           predicate_ids:
 0      skos:exactMatch
111    skos:broadMatch
Name: predicate_id, dtype: object 


In [11]:
%%time
# msdf_mondo.df[msdf_mondo.df['object_id'].str.contains('ICD')]
# "ICD10CM", "MONDO"
# msdf_mondo.df = msdf_mondo.df[(condition_1 & condition_2) | (condition_3 & condition_4)]
# msdf_mondo.df['object_label'] = msdf_mondo.df['object_id'].apply(lambda x: oi.label(x))

msdf_mondo.df['object_label'] = msdf_mondo.df['object_id'].apply(lambda x: oi.label(x))
msdf_mondo.df.head()


CPU times: user 1min 24s, sys: 6.62 s, total: 1min 30s
Wall time: 1min 32s


,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label
0,MONDO:0000001,disease or disorder,skos:exactMatch,SCTID:64572001,Unspecified,None
1,MONDO:0000001,disease or disorder,skos:exactMatch,MESH:D004194,Unspecified,None
2,MONDO:0000001,disease or disorder,skos:exactMatch,DOID:4,Unspecified,disease
3,MONDO:0000001,disease or disorder,skos:exactMatch,Orphanet:377788,Unspecified,Disease
4,MONDO:0000001,disease or disorder,skos:exactMatch,NCIT:C2991,Unspecified,Disease or Disorder


In [12]:
condition_1 = msdf_mondo.df['subject_id'].str.contains("MONDO")
condition_2 = msdf_mondo.df['object_id'].str.contains("ICD10CM")
condition_3 = msdf_mondo.df['object_id'].str.contains('|'.join((["OMIM","OMIMPS"])))
condition_4 = msdf_mondo.df['object_id'].str.contains("Orphanet")
condition_5 = msdf_mondo.df['object_id'].str.contains("DOID")
condition_6 = msdf_mondo.df['object_id'].str.contains("NCIT")

mondo_icd_df = msdf_mondo.df[condition_1 & condition_2]
mondo_omim_df = msdf_mondo.df[condition_1 & condition_3]
mondo_ordo_df = msdf_mondo.df[condition_1 & condition_4]
mondo_doid_df = msdf_mondo.df[condition_1 & condition_5]
mondo_ncit_df = msdf_mondo.df[condition_1 & condition_6]

mondo_icd_df.head()
mondo_omim_df.head()
mondo_ordo_df.head()
mondo_doid_df.head()
mondo_ncit_df.head()


,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label
4,MONDO:0000001,disease or disorder,skos:exactMatch,NCIT:C2991,Unspecified,Disease or Disorder
7,MONDO:0000004,adrenocortical insufficiency,skos:exactMatch,NCIT:C26691,Unspecified,Adrenocortical Insufficiency
18,MONDO:0000022,nocturnal enuresis,skos:exactMatch,NCIT:C118172,Unspecified,Nocturnal Enuresis
47,MONDO:0000087,polymicrogyria,skos:exactMatch,NCIT:C116936,Unspecified,Polymicrogyria
53,MONDO:0000088,precocious puberty,skos:exactMatch,NCIT:C79704,Unspecified,Precocious Puberty


In [13]:
print_prefixes(msdf_lex.df)

subject_prefixes:
 0             DOID
151307     ICD10CM
166651    ICD10WHO
169860       MONDO
425468        NCIT
458795        OMIM
474649    Orphanet
Name: subject_id, dtype: object 
           object_prefixes:
 0         MONDO
7          NCIT
8      Orphanet
11     ICD10WHO
20         DOID
37      ICD10CM
199        OMIM
Name: object_id, dtype: object 
           predicate_ids:
 0       skos:broadMatch
1       skos:exactMatch
18      skos:closeMatch
176    skos:narrowMatch
Name: predicate_id, dtype: object 


### Flipping subject_id and object_id if MONDO not subject_id prefix

**Predicate impact**

- skos:closeMatch <=> skos:relatedMatch
- skos:narrowMatch < => skos:broadMatch

In [14]:
condition_1 = msdf_lex.df['subject_id'].str.contains("MONDO")
condition_2 = msdf_lex.df['object_id'].str.contains("ICD10CM")
condition_3 = msdf_lex.df['object_id'].str.contains('|'.join((["OMIM","OMIMPS"])))
condition_4 = msdf_lex.df['object_id'].str.contains('|'.join((["ORDO","Orphanet"])))
condition_5 = msdf_lex.df['object_id'].str.contains("DOID")
condition_6 = msdf_lex.df['object_id'].str.contains("NCIT")
condition_mondo_obj = msdf_lex.df['object_id'].str.contains("MONDO")

non_mondo_subjects_df =  pd.DataFrame(msdf_lex.df[(~condition_1 & condition_mondo_obj)])
mondo_subjects_df = pd.DataFrame(msdf_lex.df[(condition_1 & ~condition_mondo_obj)])
print(len(mondo_subjects_df))
non_mondo_subjects_df.head()


44371


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,DOID:0001816,angiosarcoma,skos:broadMatch,MONDO:0003022,pediatric angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasBroadSynonym,angiosarcoma
1,DOID:0001816,angiosarcoma,skos:exactMatch,MONDO:0016982,angiosarcoma,semapv:LexicalMatching,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,mesh:d006394
2,DOID:0001816,angiosarcoma,skos:exactMatch,MONDO:0016982,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,angiosarcoma
3,DOID:0001816,angiosarcoma,skos:exactMatch,MONDO:0016982,angiosarcoma,semapv:RegularExpressionReplacement,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,angiosarcoma
4,DOID:0001816,angiosarcoma,skos:exactMatch,MONDO:0016982,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,hemangiosarcoma


In [15]:
desired_sequence = ['subject_id', 'subject_label', 'object_id',
        'predicate_id','object_label', 'mapping_justification', 'mapping_tool', 'confidence',
        'subject_match_field', 'object_match_field', 'match_string']

new_subjects_df = non_mondo_subjects_df.rename(columns={
    'subject_id':'object_id', 
    'subject_label':'object_label',
    'object_id':'subject_id', 
    'object_label':'subject_label',
    'subject_match_field': 'object_match_field',
    'object_match_field': 'subject_match_field'
    
})


new_subjects_df = new_subjects_df[desired_sequence]
new_subjects_df["predicate_id"] = new_subjects_df["predicate_id"].apply(lambda x: flip_predicate(x))
print(len(new_subjects_df))
new_subjects_df.head()


103462


,subject_id,subject_label,object_id,predicate_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,MONDO:0003022,pediatric angiosarcoma,DOID:0001816,skos:narrowMatch,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,oio:hasBroadSynonym,rdfs:label,angiosarcoma
1,MONDO:0016982,angiosarcoma,DOID:0001816,skos:exactMatch,angiosarcoma,semapv:LexicalMatching,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,mesh:d006394
2,MONDO:0016982,angiosarcoma,DOID:0001816,skos:exactMatch,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,rdfs:label,angiosarcoma
3,MONDO:0016982,angiosarcoma,DOID:0001816,skos:exactMatch,angiosarcoma,semapv:RegularExpressionReplacement,oaklib,0.800000,oio:hasExactSynonym,rdfs:label,angiosarcoma
4,MONDO:0016982,angiosarcoma,DOID:0001816,skos:exactMatch,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,hemangiosarcoma


### Combine dfs where subject_id prefix is MONDO

In [16]:
lex_df = pd.concat([mondo_subjects_df,new_subjects_df], ignore_index=True)
print(len(lex_df))

lex_df.head()

147833


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,MONDO:0000001,disease or disorder,skos:exactMatch,Orphanet:557493,disorder,semapv:LexicalMatching,oaklib,0.8,oio:hasExactSynonym,rdfs:label,disorder
1,MONDO:0000001,disease or disorder,skos:exactMatch,NCIT:C25457,Condition,semapv:LexicalMatching,oaklib,0.8,oio:hasExactSynonym,rdfs:label,condition
2,MONDO:0000001,disease or disorder,skos:exactMatch,NCIT:C156809,Medical Condition,semapv:LexicalMatching,oaklib,0.8,oio:hasExactSynonym,rdfs:label,medical condition
3,MONDO:0000004,adrenocortical insufficiency,skos:exactMatch,NCIT:C113211,Hypocortisolemia,semapv:LexicalMatching,oaklib,0.8,oio:hasExactSynonym,rdfs:label,hypocortisolemia
4,MONDO:0000022,nocturnal enuresis,skos:exactMatch,OMIM:600631,"enuresis, nocturnal, 1",semapv:LexicalMatching,oaklib,0.8,oio:hasExactSynonym,oio:hasExactSynonym,bedwetting


In [17]:
# msdf_lex.df[msdf_lex.df['object_id'].str.contains('ICD')]
condition_1 = lex_df['subject_id'].str.contains("MONDO")
condition_2 = lex_df['object_id'].str.contains("ICD10CM")
condition_3 = lex_df['object_id'].str.contains('|'.join((["OMIM","OMIMPS"])))
condition_4 = lex_df['object_id'].str.contains("Orphanet")
condition_5 = lex_df['object_id'].str.contains("DOID")
condition_6 = lex_df['object_id'].str.contains("NCIT")


mondo_icd_lex_df = lex_df[(condition_1 & condition_2)]
mondo_omim_lex_df = lex_df[(condition_1 & condition_3)]
mondo_ordo_lex_df = lex_df[(condition_1 & condition_4)]
mondo_doid_lex_df = lex_df[(condition_1 & condition_5)]
mondo_ncit_lex_df = lex_df[(condition_1 & condition_6)]

mondo_icd_lex_df.head()
mondo_omim_lex_df.head()
mondo_ordo_lex_df.head()
mondo_doid_lex_df.head()
mondo_ncit_lex_df.head()


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
1,MONDO:0000001,disease or disorder,skos:exactMatch,NCIT:C25457,Condition,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,rdfs:label,condition
2,MONDO:0000001,disease or disorder,skos:exactMatch,NCIT:C156809,Medical Condition,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,rdfs:label,medical condition
3,MONDO:0000004,adrenocortical insufficiency,skos:exactMatch,NCIT:C113211,Hypocortisolemia,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,rdfs:label,hypocortisolemia
28,MONDO:0000159,bone marrow failure syndrome,skos:exactMatch,NCIT:C165614,Bone Marrow Failure Syndrome,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,bone marrow failure syndrome
55,MONDO:0000328,hyperphosphatemia,skos:exactMatch,NCIT:C113750,Hyperphosphatemia,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,hyperphosphatemia


In [18]:
%%time
# comparison_ms_diff = compare_dataframes(msdf_mondo.df, msdf_lex.df)
# comparison_df = comparison_ms_diff.combined_dataframe
# comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_1', "MONDO_MAPPINGS")
# comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_2', "LEXMATCH")
# comparison_df.head()

# Comparisons
icd_comparison_df = compare_and_comment_df(mondo_icd_df, mondo_icd_lex_df)
omim_comparison_df = compare_and_comment_df(mondo_omim_df, mondo_omim_lex_df)
ordo_comparison_df = compare_and_comment_df(mondo_ordo_df, mondo_ordo_lex_df)
doid_comparison_df = compare_and_comment_df(mondo_doid_df, mondo_doid_lex_df)
ncit_comparison_df = compare_and_comment_df(mondo_ncit_df, mondo_ncit_lex_df)


CPU times: user 1min 40s, sys: 469 ms, total: 1min 40s
Wall time: 1min 41s


In [19]:
icd_comparison_df
omim_comparison_df
ordo_comparison_df
doid_comparison_df
ncit_comparison_df

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
15330,MONDO:0005306,ankylosing spondylitis,skos:exactMatch,NCIT:C84564,Unspecified,Ankylosing Spondylitis,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
5777,MONDO:0002203,constipation disorder,skos:exactMatch,NCIT:C37930,Unspecified,Constipation,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
51078,MONDO:0015397,oculo-auriculo-vertebral spectrum,skos:exactMatch,NCIT:C84740,Unspecified,Goldenhar Syndrome,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
62876,MONDO:0019734,juvenile polymyositis,skos:exactMatch,NCIT:C114358,Unspecified,Juvenile Polymyositis,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
50715,MONDO:0015265,bronchiolitis obliterans with obstructive pulm...,skos:exactMatch,NCIT:C62580,Unspecified,Bronchiolitis Obliterans,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
8727,MONDO:0009478,combined immunodeficiency due to DOCK8 deficiency,skos:closeMatch,NCIT:C126343,semapv:LexicalMatching,DOCK8 Deficiency,COMMON_TO_BOTH,oaklib,NaN,oio:hasRelatedSynonym,rdfs:label,dock8 deficiency
28547,MONDO:0009033,temtamy syndrome,skos:exactMatch,NCIT:C148371,Unspecified,Temtamy Syndrome,COMMON_TO_BOTH,NaN,NaN,NaN,NaN,NaN
7472,MONDO:0009033,temtamy syndrome,skos:closeMatch,NCIT:C148371,semapv:LexicalMatching,Temtamy Syndrome,COMMON_TO_BOTH,oaklib,NaN,oio:hasRelatedSynonym,rdfs:label,temtamy syndrome
61995,MONDO:0019437,enthesitis-related juvenile idiopathic arthritis,skos:exactMatch,NCIT:C119024,Unspecified,Enthesitis-Related Arthritis,COMMON_TO_BOTH,NaN,NaN,NaN,NaN,NaN


In [20]:
ncit_comparison_df['comment'].drop_duplicates()

15330    MONDO_MAPPINGS
32689          LEXMATCH
25597    COMMON_TO_BOTH
Name: comment, dtype: object

### Split into unmapped dataframes

In [21]:
unmapped_icd_df = get_unmapped_df(icd_comparison_df)
unmapped_omim_df = get_unmapped_df(omim_comparison_df)
unmapped_ordo_df = get_unmapped_df(ordo_comparison_df)
unmapped_doid_df = get_unmapped_df(doid_comparison_df)
unmapped_ncit_df = get_unmapped_df(ncit_comparison_df)

unmapped_icd_df.head()


,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
143436,MONDO:0005311,atherosclerosis,skos:exactMatch,ICD10CM:I25.1,semapv:LexicalMatching,Atherosclerotic heart disease of native corona...,LEXMATCH,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,atherosclerotic cardiovascular disease
145143,MONDO:0008104,Noonan syndrome 1,skos:closeMatch,ICD10CM:Q87.19,semapv:LexicalMatching,Other congenital malformation syndromes predom...,LEXMATCH,oaklib,NaN,oio:hasRelatedSynonym,oio:hasExactSynonym,noonan syndrome
144926,MONDO:0005345,hypospadias,skos:exactMatch,ICD10CM:Q54,semapv:RegularExpressionReplacement,Hypospadias,LEXMATCH,oaklib,0.800000,oio:hasExactSynonym,rdfs:label,hypospadias
144927,MONDO:0005345,hypospadias,skos:exactMatch,ICD10CM:Q54,semapv:LexicalMatching,Hypospadias,LEXMATCH,oaklib,0.800000,oio:hasExactSynonym,rdfs:label,hypospadias
144928,MONDO:0005345,hypospadias,skos:exactMatch,ICD10CM:Q54,semapv:LexicalMatching,Hypospadias,LEXMATCH,oaklib,0.849779,rdfs:label,rdfs:label,hypospadias


In [22]:
print_prefixes(unmapped_ncit_df)

subject_prefixes:
 26599    MONDO
Name: subject_id, dtype: object 
           object_prefixes:
 26599    NCIT
Name: object_id, dtype: object 
           predicate_ids:
 26599     skos:exactMatch
43011     skos:closeMatch
147141    skos:broadMatch
Name: predicate_id, dtype: object 


In [23]:
# Let reviewers check if this makes sense or no.
export_unmatched_exact(unmapped_icd_df, "LEXMATCH", "unmapped_icd_lex.tsv")
export_unmatched_exact(unmapped_omim_df, "LEXMATCH", "unmapped_omim_lex.tsv")
export_unmatched_exact(unmapped_ordo_df, "LEXMATCH", "unmapped_ordo_lex.tsv")
export_unmatched_exact(unmapped_doid_df, "LEXMATCH", "unmapped_doid_lex.tsv")
export_unmatched_exact(unmapped_ncit_df, "LEXMATCH", "unmapped_ncit_lex.tsv")


/var/folders/j5/bzs7jjf12j998fm88l0ssg740000gp/T/ipykernel_6499/2787498206.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predicate_id'] = df['predicate_id'].apply(lambda x: mondo_codes[x] if x in mondo_codes else x)
/var/folders/j5/bzs7jjf12j998fm88l0ssg740000gp/T/ipykernel_6499/2787498206.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmapped_exact.loc[-1] = [
/var/folders/j5/bzs7jjf12j998fm88l0ssg740000gp/T/ipykernel_6499/2787498206.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,ID,,A oboInOwl:hasDbXref,>A oboInOwl:source,>A sssom:object_label,,,,,,,
29,MONDO:0000159,bone marrow failure syndrome,MONDO:equivalentTo,NCIT:C165614,semapv:LexicalMatching,Bone Marrow Failure Syndrome,LEXMATCH,oaklib,0.849779,rdfs:label,rdfs:label,bone marrow failure syndrome
76,MONDO:0000376,respiratory system cancer,MONDO:equivalentTo,NCIT:C4571,semapv:LexicalMatching,Malignant Respiratory System Neoplasm,LEXMATCH,oaklib,0.8,oio:hasExactSynonym,rdfs:label,malignant respiratory system neoplasm
95,MONDO:0000437,cerebellar ataxia,MONDO:equivalentTo,NCIT:C26702,semapv:LexicalMatching,Ataxia,LEXMATCH,oaklib,0.8,oio:hasExactSynonym,rdfs:label,ataxia
121,MONDO:0000541,jejunal adenocarcinoma,MONDO:equivalentTo,NCIT:C181158,semapv:LexicalMatching,Jejunal Adenocarcinoma,LEXMATCH,oaklib,0.849779,rdfs:label,rdfs:label,jejunal adenocarcinoma


In [24]:
# Inspect why these are missing from SSSOM mappings
export_unmatched_exact(unmapped_icd_df, "MONDO_MAPPINGS", "unmapped_icd_mondo.tsv")
export_unmatched_exact(unmapped_omim_df, "MONDO_MAPPINGS", "unmapped_omim_mondo.tsv")
export_unmatched_exact(unmapped_ordo_df, "MONDO_MAPPINGS", "unmapped_ordo_mondo.tsv")
export_unmatched_exact(unmapped_doid_df, "MONDO_MAPPINGS", "unmapped_doid_mondo.tsv")
export_unmatched_exact(unmapped_ncit_df, "MONDO_MAPPINGS", "unmapped_ncit_mondo.tsv")

/var/folders/j5/bzs7jjf12j998fm88l0ssg740000gp/T/ipykernel_6499/2787498206.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predicate_id'] = df['predicate_id'].apply(lambda x: mondo_codes[x] if x in mondo_codes else x)
/var/folders/j5/bzs7jjf12j998fm88l0ssg740000gp/T/ipykernel_6499/2787498206.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmapped_exact.loc[-1] = [
/var/folders/j5/bzs7jjf12j998fm88l0ssg740000gp/T/ipykernel_6499/2787498206.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,ID,,A oboInOwl:hasDbXref,>A oboInOwl:source,>A sssom:object_label,,,,,,,
5,MONDO:0000001,disease or disorder,MONDO:equivalentTo,NCIT:C2991,Unspecified,Disease or Disorder,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
104,MONDO:0000147,polyposis,MONDO:equivalentTo,NCIT:C4089,Unspecified,Polyposis,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
471,MONDO:0000371,oral cavity carcinoma in situ,MONDO:equivalentTo,NCIT:C4587,Unspecified,Stage 0 Oral Cavity Cancer AJCC v6 and v7,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
477,MONDO:0000372,pharynx carcinoma in situ,MONDO:equivalentTo,NCIT:C4942,Unspecified,Stage 0 Pharyngeal Cancer,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [25]:
# # Add distances
# # Add Levenshtein distance [ 0: Perfect match]
# add_distance(unmapped_mondo_exact, "levenshtein_dist", textdistance.levenshtein.distance)
# # Add Jaccard Index [ 0: Perfect match]
# add_distance(unmapped_mondo_exact, "jaccard_index", textdistance.jaccard.distance)
# # Add Monge-Elkan Distance [ 0: Perfect match]
# add_distance(unmapped_mondo_exact, "monge_elkan", textdistance.monge_elkan.distance)
# unmapped_mondo_exact.to_csv(join(dir_name, "unmapped_mondo_exact.tsv"), sep='\t', index = False)
# unmapped_mondo_exact.head()


In [26]:
# combined_msdf_icd = make_msdf(icd_comparison_df, msdf_lex.prefix_map, msdf_lex.metadata)
# combined_msdf_omim = make_msdf(omim_comparison_df, msdf_lex.prefix_map, msdf_lex.metadata)
# combined_msdf_ordo = make_msdf(ordo_comparison_df, msdf_lex.prefix_map, msdf_lex.metadata)
# combined_msdf_doid = make_msdf(doid_comparison_df, msdf_lex.prefix_map, msdf_lex.metadata)
# combined_msdf_ncit = make_msdf(ncit_comparison_df, msdf_lex.prefix_map, msdf_lex.metadata)

# combined_msdf_icd.df.head()

In [27]:
# %%time
# df_dict = split_dataframe(combined_msdf)

In [28]:
# mondo_icd_list = [x for x in list(df_dict.keys()) if 'mondo' in x and "icd10" in x]
# mondo_icd_list

In [29]:
# for match in mondo_icd_list:
#     fn = match + ".tsv"
#     df_dict[match].df.to_csv(join(dir_name,fn), sep='\t', index = False)

In [30]:
# df_dict['mondo_exactmatch_icd10cm'].df